In [1]:
%load_ext autoreload
%autoreload 2

from kasearch import AlignSequences, SearchDB, PrepareDB

## Prepare query sequence (sequence to search with)

In [2]:
raw_queries = [
    'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS',
    'EVQLQQSGTVLARPGASVKMSCEASGYTFTNYWMHWVKQRPGQGLEWIGAIYPGNSDTSYIQKFKGKAKLTAVTSTTSVYMELSSLTNEDSAVYYCTLYDGYYVFAYWGQGTLVTVSA',
]

In [3]:
query_db = AlignSequences(raw_queries, # Sequences as strings to align.
                          n_jobs=1     # Allocated number for jobs/threads for the search.
                         )
query_db.db.aligned_seqs

array([[81, 86, 75,  0, 76, 81, 69, 83, 71, 65,  0, 69, 76, 65, 82, 80,
        71, 65, 83, 86, 75, 76, 83, 67, 75, 65, 83, 71, 89, 84, 70,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 84, 78, 89, 87, 77, 81,  0, 87,
        86, 75, 81,  0, 82,  0, 80,  0, 71,  0, 81,  0,  0, 71,  0, 76,
        68,  0, 87, 73, 71, 65, 73, 89, 80, 71,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 68, 71, 78, 84, 82, 89,  0,  0, 84,  0,  0,
        72,  0,  0, 75, 70,  0,  0, 75,  0,  0,  0, 71, 75, 65, 84, 76,
        84, 65,  0, 68,  0,  0,  0, 75,  0, 83,  0,  0, 83, 83,  0,  0,
         0,  0, 84,  0, 65, 89, 77, 81, 76, 83, 83, 76, 65, 83,  0, 69,
        68, 83, 71, 86, 89, 89, 67, 65, 82, 71, 69, 71, 78,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 89, 65, 87, 70, 65, 89, 87, 71,  0, 81,
        71, 84, 84, 86, 84, 86, 83, 83],
       [69, 86, 81,  0, 76, 81, 81, 83, 71, 84,  0, 86, 76, 65, 82, 80,
        71, 65, 83, 86,

--------------
## Initiate search class 

- Default will download a prepared version of OAS to search against
- Alternatively, you can give it the path for a custom database to search against. (See below for how to create a custom database).

In [4]:
oasdb = SearchDB(database_path='oasdb-small', # Path to your database. Default will be to download a small prepared version of OAS.
                 allowed_chain='Heavy',       # Search against a specific chain. Default is any chain.
                 allowed_species='Rat',       # Search against a specific species. Default is any species.
                 n_jobs=1                     # Allocated number for jobs/threads for the search.
                )

-----------
## Run search

A search against all of OAS takes ~2min for each sequence.


In [5]:
%%time 
oasdb.search(query_db.db.aligned_seqs, # Input can only be a single aligned sequence at a time.
             keep_best_n=4,               # You can define how many most similar sequences to return
             reset_best=True              # In cases where you want to search with the same sequence against multiple databases, you might want to not reset_best. (True is default)
            )

CPU times: user 618 ms, sys: 7.97 ms, total: 626 ms
Wall time: 1.58 s


### Get N best identities

Identities of the most similar sequence (1st column), CDRs (2nd column) and CDR3 (3rd column) in all of OAS, can be fetched from the object with the bellow command.

In [6]:
oasdb.current_best_identities

array([[[0.7660823 , 0.6785714 , 0.75      ],
        [0.7545546 , 0.6785714 , 0.75      ],
        [0.746353  , 0.64285713, 0.75      ],
        [0.7458163 , 0.64285713, 0.75      ]],

       [[0.6864407 , 0.5555556 , 0.6363636 ],
        [0.68381065, 0.5555556 , 0.6363636 ],
        [0.6808634 , 0.5555556 , 0.6363636 ],
        [0.6779311 , 0.5185185 , 0.54545456]]], dtype=float32)

### Get ID's of sequences with the N best identities

ID's of the most similar sequence (1st column), CDRs (2nd column) and CDR3 (3rd column) in all of OAS, can be fetched from the object with the bellow command. 

With ExtractMetadata, these ID's can be used to extract the sequence and its meta data (species, isotype, whole sequence etc.).

In [7]:
oasdb.current_best_ids

array([[[[ 13619,  44954],
         [ 13590,  48248],
         [ 13572,    884]],

        [[ 13591,  13614],
         [ 13572,  68747],
         [ 13608,  80523]],

        [[ 13591,  29438],
         [ 13605,  12566],
         [ 13605,  61535]],

        [[ 13608,  35406],
         [ 13597, 147730],
         [ 13605,   7611]]],


       [[[ 13615,  18993],
         [ 13602,  42681],
         [ 13573,  31570]],

        [[ 13619,  44954],
         [ 13615,   4042],
         [ 13569,   7078]],

        [[ 13602,  83775],
         [ 13602,  32964],
         [ 13604,  12380]],

        [[ 13625,  19612],
         [ 13602,  20418],
         [ 13569,   1035]]]], dtype=int32)

---------
## Extract the meta data from matched sequences


The meta data for the N best of the different regions, can be extracted with the following n_regions.
- 0: whole sequence
- 1: CDRs
- 2: CDR3

e.g. "oasdb.get_meta\[n_query0,0\]\" returns meta data for the N best whole sequence identities.

NB: The column "sequence_alignment_aa" holds the antibody sequence.

In [8]:
n_best_sequences = oasdb.get_meta(n_query = 0,          # Which query to extract meta data from
                                  n_region = 0,         # Which region to extract meta data from
                                  n_sequences = 'all',  # Number of sequences to extract (default is all, which is keep_best_n)
                                  n_jobs=10             # Allocated number for jobs/threads for the extraction
                                 )
n_best_sequences

,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,d_call,...,BType,Subject,Disease,Vaccine,Longitudinal,Chain,Unique sequences,Isotype,Total sequences,Identity
0,CGCAGGAGGGTGTTTACTCTCAGGTCCAGCTGCAACAATCTGGGGC...,H,F,T,F,T,F,T,IGHV1-28*01,NaN,...,Unsorted-B-Cells,no,None,HuD,no,Heavy,156152,IGHG,438157,0.766082
1,AAGCGGTGTCTACTCCCAGGTTCAGCTGCAGCAATCTGGGGCTGAG...,H,F,T,F,T,T,T,IGHV1-28*01,IGHD1-12*02,...,Unsorted-B-Cells,no,None,HuD,no,Heavy,92608,IGHA,123653,0.754555
2,TAACGGTGTCTACTCTCAGGTTCAGCTGCAGCAATCTGGGGCTGAG...,H,F,T,F,T,F,T,IGHV1-28*01,IGHD1-12*02,...,Unsorted-B-Cells,no,None,HuD,no,Heavy,92608,IGHA,123653,0.746353
3,ATTCAGGTGTTCACTCTCAGGTTCAGCTGCAGCAATCTGGGGCTGA...,H,F,T,F,T,T,T,IGHV1-28*01,IGHD4-3*01,...,Unsorted-B-Cells,no,None,HuD,no,Heavy,433065,IGHG,757476,0.745816


----------
## Create custom database


To create your own database you first need to create a csv file in the OAS format. For an example file, look at data/custom-data-example.csv. This file consists of a dictionary containing the metadata in the first line and then rows of the individual sequences afterwards. Only the Species and Chain is strictly needed in the metadata, and only the amino acids sequence of the antibodies is required for each antibody sequence.

### 1. Format your data into OAS files

In [9]:
import json
import os
import pandas as pd

from kasearch.species_anarci import number
from kasearch.merge_db import merge_files

In [10]:
metadata = {"Species":"Human", "Chain":"Heavy"}
metadata = pd.Series(name=json.dumps(metadata), dtype='object')
seqsdata = pd.DataFrame([["EVQLVESGGGLAKPGGSLRLHCAASGFAFSSYWMNWVRQAPGKRLEWVSAINLGGGLTYYAASVKGRFTISRDNSKNTLSLQMNSLRAEDTAVYYCATDYCSSTYCSPVGDYWGQGVLVTVSS"],
                          ["EVQLVQSGAEVKRPGESLKISCKTSGYSFTSYWISWVRQMPGKGLEWMGAIDPSDSDTRYNPSFQGQVTISADKSISTAYLQWSRLKASDTATYYCAIKKYCTGSGCRRWYFDLWGPGT"]
                         ], columns = ['sequence_alignment_aa'])

In [11]:
save_file = "../data/custom-data-example-2.csv"
metadata.to_csv(save_file, index=False)
seqsdata.to_csv(save_file, index=False, mode='a')

### 2. Turn your OAS formatted files into a custom database

After creating all the files you want to include in the new database, you can run the following code to create the database.

In [12]:
db_folder = "../data/my_db"
db_files = ['../data/custom-data-example.csv']

In [13]:
newDB = PrepareDB(db_path=db_folder)

db_dict = {}
for num, data_unit_file in enumerate(db_files):

    db_dict[num] = data_unit_file
    
    metadata = json.loads(','.join(pd.read_csv(data_unit_file, nrows=0).columns))
    seqsdata = pd.read_csv(data_unit_file, header=1, usecols=['sequence_alignment_aa']).iloc[:,0].values
    numbered_sequences = [number(sequence, allowed_species=None)[0] for sequence in seqsdata]

    newDB.prepare_database(numbered_sequences, 
                           file_id=num, 
                           chain=metadata['Chain'], 
                           species=metadata['Species'])
    
newDB.save_database()

merge_files(db_folder)

with open(os.path.join(db_folder, "id_to_study.txt"), "w") as handle: handle.write(str(db_dict))

### 3. Initiate the search class with your custom database


In [14]:
mydb = SearchDB(database_path=db_folder,      # Path to your database. Default will be to download a prepared version of OAS.
                 allowed_chain='Heavy',       # Search against a specific chain. Default is any chain.
                 allowed_species='Any',       # Search against a specific species. Default is any species.
                 n_jobs=1                     # Allocated number for jobs/threads for the search.
                )